In [1]:
from tensorflow.keras.models import load_model

model_resnet = load_model('resnet_model.h5')


In [2]:
print("Class distribution in training set:")
print(train_generator.class_indices)

from collections import Counter

counter = Counter(train_generator.classes)
print(f"Training Class Distribution: {counter}")


Class distribution in training set:


NameError: name 'train_generator' is not defined

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your dataset directories
train_dir = 'dataset/train'
val_dir = 'dataset/validation'

# Initialize ImageDataGenerator with augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# For validation/test data, only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

# Create the generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 2736 images belonging to 2 classes.
Found 342 images belonging to 2 classes.


In [5]:
from collections import Counter

# Check class indices (labels)
print("Class distribution in training set:")
print(train_generator.class_indices)

# Check class distribution
counter = Counter(train_generator.classes)
print(f"Training Class Distribution: {counter}")


Class distribution in training set:
{'Monkeypox': 0, 'Non_Monkeypox': 1}
Training Class Distribution: Counter({1: 1512, 0: 1224})


In [7]:
# Compute class weights
from sklearn.utils import class_weight
import numpy as np
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights_dict = dict(enumerate(class_weights))

# Train the model with class weights
history = model_resnet.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    class_weight=class_weights_dict
)


Epoch 1/10

85/85 [==============================] - 438s 5s/step - loss: 0.0084 - accuracy: 0.9993 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 2/10
85/85 [==============================] - 393s 5s/step - loss: 0.0067 - accuracy: 0.9996 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 3/10
85/85 [==============================] - 392s 5s/step - loss: 0.0094 - accuracy: 0.9978 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 4/10
85/85 [==============================] - 394s 5s/step - loss: 0.0078 - accuracy: 0.9993 - val_loss: 0.0059 - val_accuracy: 1.0000
Epoch 5/10
85/85 [==============================] - 393s 5s/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 6/10
85/85 [==============================] - 394s 5s/step - loss: 0.0062 - accuracy: 0.9993 - val_loss: 0.0066 - val_accuracy: 0.9969
Epoch 7/10
85/85 [==============================] - 394s 5s/step - loss: 0.0087 - accuracy: 0.9993 - val_loss: 0.0043 - val_accuracy: 1.0000
Epoch 8/10
8

In [8]:
# Save the entire model to a file
model_resnet.save('resnet_model_n.h5')


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data augmentation for testing (just rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',  # Path to the test dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important: do not shuffle if you want to compare true labels to predictions
)


Found 342 images belonging to 2 classes.


In [11]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on validation/test set
predictions = model_resnet.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)

# Generate a classification report
print("Classification Report:")
print(classification_report(test_generator.classes, predicted_classes, target_names=['Monkeypox', 'Non_Monkeypox']))

# Generate a confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_generator.classes, predicted_classes))


11/11 [==============================] - 14s 1s/step
Classification Report:
               precision    recall  f1-score   support

    Monkeypox       1.00      0.99      1.00       153
Non_Monkeypox       0.99      1.00      1.00       189

     accuracy                           1.00       342
    macro avg       1.00      1.00      1.00       342
 weighted avg       1.00      1.00      1.00       342

Confusion Matrix:
[[152   1]
 [  0 189]]


In [26]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Load your trained model
model_resnet = tf.keras.models.load_model('resnet_model_n.h5')

# Function to predict Monkeypox from an image
def predict_monkeypox(img_path):
    # Load the image with the same target size as the training images
    img = image.load_img(img_path, target_size=(224, 224))
    
    # Convert the image to an array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the input shape (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize the image data (if you used rescaling during training)
    img_array /= 255.0
    
    # Make a prediction
    prediction = model_resnet.predict(img_array)
    
    # Convert the prediction to a binary class label
    if prediction < 0.5:
        print("Prediction: Monkeypox")
    else:
        print("Prediction: Not Monkeypox")

# Example usage
img_path = 'measles11.jpg'
predict_monkeypox(img_path)


1/1 [==============================] - 1s 1s/step
Prediction: Not Monkeypox
